# CBF Tutorial - Part 3: Higher-Order Barrier Functions

Authors: Bardh, Tom and Shakiba

## In this notebook, we consider higher-order barrier functions.

We start with **limber_uniz_1cle** system, which has the following dynamics:
$$
\begin{align*}
  \dot{x} = f(x) + g(x)*u \\

  \begin{bmatrix} \dot{xr_0} \\ \dot{xr_1} \\ \dot{xr_2} \end{bmatrix} = \begin{bmatrix} cos(xr_2) \\ sin(xr_2) \\ 0 \end{bmatrix} + \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix} * u \tag{1}\\
\end{align*}
$$

where the input to the system $u=\omega$ is the angular velocity.

Given an initial position $x^{init}$, our goal is to control this system to a goal location $x^{goal}$.
$
\begin{align*}
x^{init} = \begin{bmatrix} x^{init}_0 \\  x^{init}_1 \end{bmatrix} \qquad	x^{goal} = \begin{bmatrix} x^{goal}_0 \\  x^{goal}_1 \end{bmatrix} \tag{2}\\
\end{align*}
$
However, the system should avoid certain areas in the workspace (more on this later).

One way to solve this problem is to pose it as an optimization problem. Specifically, we would like to solve the following problem:
$
\begin{align*}
\min_{u} \quad & ||u_{ref} - u||\\
\textrm{s.t.} \quad & \text{CBF constraints to be added later} \tag{3}\\
\end{align*}
$
So, we will try to minimize the distance between the decission variable $u$ and reference controller $u_{ref}$, while satisfying some constraints. For now, let us disregard the constraint and focus on designing a reference controller.

## Reference controller

We will define a very simple reference controller as follows
$
\begin{align*}
u_{ref} = k \cdot \arctan{(\frac{x^{goal}_0-xr_0}{x^{goal}_1-xr_1})} - xr_2 \tag{4}
\end{align*}
$
This is a two dimensional vector that takes the difference between the current system state (or location) and the goal location and multiply it with a gain factor $k$. This **simple** reference controller will take us from an initial location to the goal location. We can say it is a typical [proportional controller](https://en.wikipedia.org/wiki/Proportional_control). Next, we need to make sure that the system does not enter undesired states.

## Control Barrier Function (CBF) as Constraints

We assume that the undesired regions in the workspace can be defined with one or more [ellipses](https://en.wikipedia.org/wiki/Ellipse). In two dimensions, the region inside an ellypsis can be using the standard form with the following inequality
$
\begin{align*}
\frac{z^2_0}{a^2} + \frac{z^2_1}{b^2}\leq 1 \tag{5}
\end{align*}
$
where $z_0,z_1$ define the origin and $a,b$ define the major, minor axes of the ellipse, respectively.

In our case, we would like to check whether our system is in the undesired region and therefore we modify the afforementined requirement
$
\begin{align*}
B(x) = \frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1 \tag{6}
\end{align*}
$

In order to generate CBF, we need a more general theory of CBF. A CBF (AMES,2019) is defined as:

$
\begin{align}
L_fB(x) + L_gB(x)u + \alpha(B(x)) \ge 0
\end{align}
$

Where $L_fB(x) = \frac{\partial B}{\partial x}^Tf(x)$ and $L_gB(x) = \frac{\partial B}{\partial x}^Tg(x)$ are the first order Lie derivatives of the system. This works well for first order systems, where the control input appears in the first derivative of $B$.

However, in our example, as we shall see, this function has a relative degree of 2 with respect to the system under consideration, since the first time that the control $u$ appears in the derivatives of B along the system dynamics is in its $2^{nd}$ derivative. In this case:

$
\begin{align}
L^2_fB(x) + L_gL_fB(x)u + \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) \ge 0
\end{align}
$

First,
$$
\begin{align}
\dot{B}(x) = \frac{\partial{B}}{\partial{x}} \cdot \frac{\partial{x}}{\partial{t}} = \begin{bmatrix} (-2z_0 + 2xr_0)/a^2] \\ (-2z_1 + 2xr_1)/b^2] \\ 0 \end{bmatrix}^T * \dot{x}
\end{align}
$$

Then, we caluclate
$$
\begin{align}
L^2_fB(x) = ((-2z_0 + 2xr_0)/a^2 + 2cos(xr_2)/a^2)*cos(xr_2) + ((-2z_1 + 2xr_1)/b^2 + 2sin(xr_2)/b^2)*sin(xr_2)
\end{align}
$$
<!-- L^2_fB(x) = (\alpha_1*(-2z_0 + 2xr_0)/a^2 + 2cos(xr_2)/a^2)*cos(xr_2) + (\alpha_1*(-2z_1 + 2xr_1)/b^2 + 2sin(xr_2)/b^2)*sin(xr_2) -->
After, we calculate
$$
\begin{align}
L_gL_fB(x)u = ((-2z_1 + 2xr_1)*cos(xr_2)/b^2 - (-2z_0 + 2xr_0)*sin(xr_2)/a^2)*\dot{xr_2}
\end{align}
$$

Here, we notice that our control input $\dot{xr_2}$ appears in the expression and we can therefore continue in the next step of synthesizing a controller.

## Posing the control problem as a quadratic program

We can now combine the cost function with the constraints to form an optimization problem.
$
\begin{align*}
\min_{u} \quad & ||u_{ref} - u||\\ \textrm{s.t.} \quad & L_gL_fB(x)u  \ge -L^2_fB(x) - \alpha_2( \dot{B}(x) + \alpha_1 B(x) ) 
\end{align*}
$

This problem can be posed as a [quadratic problem](https://en.wikipedia.org/wiki/Quadratic_programming) of the form
$
\begin{align*}
\min_{\mathbf{x}} \quad & \frac{1}{2}\mathbf{x}^T P\mathbf{x} + q^T\mathbf{x} \\
\textrm{s.t.} \quad & G\mathbf{x} \le h \tag{13}
\end{align*}
$
In our case, $\mathbf{x} = \dot{x}$ is the decision variable,
Where

$
\begin{align*}
P = I_2 \tag{14}
\end{align*}
$
$
\begin{align*}
q = \begin{bmatrix} -u_{ref0} \\  -u_{ref1} \end{bmatrix} \tag{15}
\end{align*}
$
$
\begin{align*}
G = \begin{bmatrix} \frac{-2}{a^2}(x_0 - z_0) \\
    \frac{-2}{b^2}(x_1 - z_1)\end{bmatrix} \tag{16}
\end{align*}
$
$
\begin{align*}
h = (\frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1) \tag{17}
\end{align*}
$
(The expansion is [here](./CBF2quadratic.ipnyb))

Here, one of advantage of this methodology is that instance of controller is not needed, as we can see (15) referes only result of controller reference $u_{ref}$. In another word, the CBF can work as an observer in whole controll system.

## Code implementation

First, we import the necessary packages for our example.
